In [2]:
import numpy as np
from __future__ import print_function
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
pickle_file = 'notMNIST.pickle'
try:
    f = open(pickle_file,'rb')
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    
except Exception as e:
    print(e)
    raise

train_dataset.shape

(200000, 28, 28)

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset,labels):
    dataset = dataset.reshape((-1,image_size*image_size)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset,labels

train_dataset, train_labels = reformat(train_dataset,train_labels)
valid_dataset, valid_labels = reformat(valid_dataset,valid_labels)
test_dataset, test_labels = reformat(test_dataset,test_labels)

print("Training data ", train_dataset.shape, train_labels.shape)
print("Validation data ", valid_dataset.shape, valid_dataset.shape)
print("Testing data ", test_dataset.shape, test_labels.shape)

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Training data  (200000, 784) (200000, 10)
Validation data  (10000, 784) (10000, 784)
Testing data  (10000, 784) (10000, 10)


In [61]:
num_hidden_units = 1024
batch_size = 128


graph = tf.Graph()

with graph.as_default():
    tensor_training_data = tf.placeholder(np.float32,shape=(batch_size, (image_size*image_size)))
    tensor_training_labels = tf.placeholder(np.float32,shape=(batch_size,num_labels))
    tensor_validation_data = tf.constant(valid_dataset)
    tensor_validation_labels = tf.constant(valid_labels)
    tensor_testing_data = tf.constant(test_dataset)
    tensor_testing_labels = tf.constant(test_labels)
    
    weights_10 = tf.Variable(tf.truncated_normal([(image_size*image_size),num_hidden_units]))
    biases_0 = tf.Variable(tf.random_normal([num_hidden_units],seed=128))
    
    weights_21 = tf.Variable(tf.truncated_normal([num_hidden_units,num_labels]))
    biases_1 = tf.Variable(tf.random_normal([num_labels],seed=128))
    
    hidden_layer = tf.nn.dropout(tf.nn.relu(tf.matmul(tensor_training_data, weights_10)+biases_0),keep_prob=0.5)
    logits = tf.matmul(hidden_layer,weights_21) + biases_1
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tensor_training_labels, logits=logits))
    optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
    
    training_predictions = tf.nn.softmax(logits)
    validation_predictions =tf.nn.softmax(
        tf.matmul(
            tf.nn.relu(
                tf.matmul(tensor_validation_data,weights_10)+biases_0
                    ),weights_21)+biases_1)
    testing_predictions =tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tensor_testing_data,weights_10)+biases_0),weights_21)+biases_1)
    

In [62]:
num_steps = 3001
training_acc = []
validation_acc = []
loss_values = []
steps_count = []

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_training_data = train_dataset[offset:(offset+batch_size),:]
        batch_training_labels = train_labels[offset:(offset+batch_size),:]
        feed_dict = {tensor_training_data : batch_training_data, tensor_training_labels : batch_training_labels}
        
        _,l,predictions = session.run([optimizer,loss,training_predictions], feed_dict=feed_dict)
        
        if step%500 == 0:
            print("Loss at step %d: %f" % (step,l))
            loss_values.append(l)
            t_acc = accuracy(predictions,batch_training_labels)
            v_acc = accuracy(validation_predictions.eval(),valid_labels)
            training_acc.append(t_acc)
            validation_acc.append(v_acc)
            steps_count.append(step)
            print("Training accuracy: %.1f%%" % t_acc)
            print("Validation accuracy: %.1f%%" % v_acc)
    print("Testing accuracy: %.1f%%" % accuracy(testing_predictions.eval(),test_labels))

Initialized
Loss at step 0: 494.785339
Training accuracy: 3.9%
Validation accuracy: 8.7%
Loss at step 500: 75.944183
Training accuracy: 71.1%
Validation accuracy: 80.2%
Loss at step 1000: 67.614471
Training accuracy: 74.2%
Validation accuracy: 81.8%
Loss at step 1500: 49.804466
Training accuracy: 77.3%
Validation accuracy: 82.6%
Loss at step 2000: 46.611912
Training accuracy: 77.3%
Validation accuracy: 82.8%
Loss at step 2500: 43.984276
Training accuracy: 71.9%
Validation accuracy: 83.2%
Loss at step 3000: 40.199646
Training accuracy: 78.1%
Validation accuracy: 83.7%
Testing accuracy: 90.2%
